In [1]:
import numpy as np 
import pandas as pd
import csv

import time
import datetime as dt

### Function section

In [13]:
def convert_date(w):
    """ convert a date 'mm/jj/yyyy' to a format : 'yyyymmjj' """
    print(w)
    w_length = len(w)
    year = w[w_length - 4:]
    day = w[w_length - 7 : w_length - 5]

    if w_length == 10 :
        month = w[:2]
    else :
        month = '0' + w[0]
    f = year + month + day
     
    return f

def convert_time(w):
    
    """ convert a time '3:20:40.50 PM' to a format : 'hhmmssddd' """

    a = w[-2:]
    h = w[0]
    m = w[2:4]
    s = w[5:7]
    millis = w[8:10] + '0'

    if a == 'PM':

        hour = int(h) + 12
        h = str(hour)

    
    f = h + m + s + millis
    
    return f 

def convert_UTC(date,t):
    
    """ Merge date 'yyyymmjj' and a time 'hhmmssddd' to UTC time 'yyyymmjj'T'hhmmssddd'Z format, used by Dany Dumont team"""
    
    year = int(date[:4])
    month = int(date[4:6])
    day = int(date[6:8])

    hour = int(t[:2])
    minute = int(t[2:4])
    sec = int(t[4:6])
    millisec = t[6:]
    print(millisec)
    # convert a time year, month, day, hour, min, sec, microsec to a timme in seconds since the epoch
    t = dt.datetime(year,month,day,hour,minute,sec).timestamp()

    # convert a time to UTC time 
    UTC_t = time.gmtime(t)
    
    y_txt = str(UTC_t[0])
    UTC_txt = y_txt
    
    for i in range(1,6):
        
        a_txt = str(UTC_t[i])
        if UTC_t[i] < 10:
            a_txt = '0' + a_txt 
        
        UTC_txt += a_txt     
        
        if i == 2:
            UTC_txt += 'T'
        
    UTC_txt += millisec + 'Z'
    
    return UTC_txt
    

def get_timeline_row(data,idx,comments,facq):
    """ This function takes the index of a given panda DataFrame, and builds a row to be written in a timeline csv file"
    The timeline has the following headers :
    'Index','Objet','T_0','T_F','X','Y','Z','Latitude','Longitude','Elevation','Commentaire','Facq','Theta'
    
    The function takes as arguments : 
    - data, a pandas DataFrame
    - idx, the index of the DataFrame in which we are interested
    - comments, envetual comments to add, must be a string
    - facq, acquisition frequency, must be a float
    """
    # Get the date and time
    date = data['CUSTOM.date [local]'][idx]
    t = data['CUSTOM.updateTime [local]'][idx]
    
    # Convert date and time in suitable formats
    date = convert_date(date)
    t = convert_time(t)
    
    # Convert date and time in UTC format
    UTC_t = convert_UTC(date,t)
    
    # Get other variables
    Z = float(data['OSD.height [ft]'][idx_start])*0.308
    latitude = data['OSD.latitude'][idx_start]
    longitude = data['OSD.longitude'][idx_start]
    elevation = float(data['OSD.altitude [ft]'][idx_start])*0.308
    theta = data['GIMBAL.pitch'][idx_start]

    # Build a line to be written in a csv 
    line_csv = ['','',UTC_t,'','','',Z,latitude,longitude,elevation,comments,facq,theta]
    return line_csv
    
    
    
    

In [5]:
path = 'G:/Rimouski_2024/Data/2024/0211/Drones/bernache/flightrecords/DJIFlightRecord_2024-02-11_[15-26-43].csv'

data = pd.read_csv(path,header = 1, low_memory = False)
data

,CUSTOM.date [local],CUSTOM.updateTime [local],OSD.flyTime,OSD.flyTime [s],OSD.latitude,OSD.longitude,OSD.height [ft],OSD.heightMax [ft],OSD.vpsHeight [ft],OSD.altitude [ft],...,SERIAL.flightController,SERIAL.camera,SERIAL.gimbal,SERIAL.rc,SERIAL.battery,APPGPS.latitude,APPGPS.longitude,APPGPS.accuracy,APP.tip,APP.warning
0,2/11/2024,3:26:43.99 PM,0m 0.0s,0.0,48.251745,-70.091148,0.0,0.0,0.3,0,...,1581F67QC23680149Q44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Flight mode changed to P-GPS (Brake).,NaN
1,2/11/2024,3:26:44.11 PM,0m 0.1s,0.1,48.251745,-70.091148,0.0,0.0,0.3,0,...,1581F67QC23680149Q44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data Recorder File Index is 28.; Set Return to...,NaN
2,2/11/2024,3:26:44.21 PM,0m 0.2s,0.2,48.251744,-70.091148,0.0,0.0,0.3,0,...,1581F67QC23680149Q44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L'appareil n'est pas en vol. Systèmes optiques...,NaN
3,2/11/2024,3:26:44.32 PM,0m 0.3s,0.3,48.251744,-70.091148,0.0,0.0,0.3,0,...,1581F67QC23680149Q44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2/11/2024,3:26:44.42 PM,0m 0.4s,0.4,48.251744,-70.091148,0.0,0.0,0.3,0,...,1581F67QC23680149Q44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7446,2/11/2024,3:39:18.29 PM,12m 34.3s,754.3,48.251750,-70.091118,2.2,312.0,0.3,2,...,1581F67QC23680149Q44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7447,2/11/2024,3:39:18.49 PM,12m 34.4s,754.4,48.251750,-70.091118,2.2,312.0,0.3,2,...,1581F67QC23680149Q44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7448,2/11/2024,3:39:18.59 PM,12m 34.5s,754.5,48.251750,-70.091118,2.2,312.0,0.3,2,...,1581F67QC23680149Q44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7449,2/11/2024,3:39:18.59 PM,12m 34.6s,754.6,48.251750,-70.091118,2.2,312.0,0.3,2,...,1581F67QC23680149Q44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.shape
print(data.head())

  CUSTOM.date [local] CUSTOM.updateTime [local] OSD.flyTime  OSD.flyTime [s]  \
0           2/11/2024             3:26:43.99 PM     0m 0.0s              0.0   
1           2/11/2024             3:26:44.11 PM     0m 0.1s              0.1   
2           2/11/2024             3:26:44.21 PM     0m 0.2s              0.2   
3           2/11/2024             3:26:44.32 PM     0m 0.3s              0.3   
4           2/11/2024             3:26:44.42 PM     0m 0.4s              0.4   

   OSD.latitude  OSD.longitude  OSD.height [ft]  OSD.heightMax [ft]  \
0     48.251745     -70.091148              0.0                 0.0   
1     48.251745     -70.091148              0.0                 0.0   
2     48.251744     -70.091148              0.0                 0.0   
3     48.251744     -70.091148              0.0                 0.0   
4     48.251744     -70.091148              0.0                 0.0   

   OSD.vpsHeight [ft]  OSD.altitude [ft]  ...  SERIAL.flightController  \
0                 

In [23]:
data.shape

(7451, 186)

In [26]:
data.columns

Index(['CUSTOM.date [local]', 'CUSTOM.updateTime [local]', 'OSD.flyTime',
       'OSD.flyTime [s]', 'OSD.latitude', 'OSD.longitude', 'OSD.height [ft]',
       'OSD.heightMax [ft]', 'OSD.vpsHeight [ft]', 'OSD.altitude [ft]',
       ...
       'SERIAL.flightController', 'SERIAL.camera', 'SERIAL.gimbal',
       'SERIAL.rc', 'SERIAL.battery', 'APPGPS.latitude', 'APPGPS.longitude',
       'APPGPS.accuracy', 'APP.tip', 'APP.warning'],
      dtype='object', length=186)

In [27]:
data['CAMERA.isVideo']

0       False
1       False
2       False
3       False
4       False
        ...  
7446     True
7447     True
7448     True
7449     True
7450     True
Name: CAMERA.isVideo, Length: 7451, dtype: bool

### Detect start of the video 

In [7]:
mask = np.where(data['CAMERA.isVideo'] == 1)
idx_start = np.min(mask)
idx_start

1230

In [8]:
data['CUSTOM.date [local]']

0       2/11/2024
1       2/11/2024
2       2/11/2024
3       2/11/2024
4       2/11/2024
          ...    
7446    2/11/2024
7447    2/11/2024
7448    2/11/2024
7449    2/11/2024
7450    2/11/2024
Name: CUSTOM.date [local], Length: 7451, dtype: object

### Get relevant variables

In [108]:
relevant_heads = ['CUSTOM.date [local]','CUSTOM.updateTime [local]','OSD.latitude','OSD.longitude','OSD.height [ft]','OSD.altitude [ft]','GIMBAL.pitch']

line = []
for head in relevant_heads :
    
    word = data[head][idx_start]
    print(head)
    print(word)
    if head == 'CUSTOM.date [local]':
        
        word = convert_date(word)
        
    if head == 'CUSTOM.updateTime [local]':
#         new_word = time.gmtime(word)
#         print(new_word)
        word = convert_time(word)
    
    line.append(word)
    
    
print(line)



CUSTOM.date [local]
2/11/2024
2/11/2024
CUSTOM.updateTime [local]
3:28:48.53 PM
OSD.latitude
48.2517258135521
OSD.longitude
-70.0911424895404
OSD.height [ft]
49.2
OSD.altitude [ft]
49
GIMBAL.pitch
-90.0
['20240211', '152848530', 48.2517258135521, -70.0911424895404, 49.2, 49, -90.0]


In [166]:
date = line[ relevant_heads == 'CUSTOM.date [local]']

mask = np.where( np.array(relevant_heads) == 'CUSTOM.date [local]')[0]
date = line[mask[0]]

mask = np.where( np.array(relevant_heads) == 'CUSTOM.updateTime [local]')[0]
t = line[mask[0]]
print(t)
# date_formated = date + t
# print(date_formated)

# test = line[np.where(relevant_heads == 'CUSTOM.date [local]')]

152848530


### Convert a date/time to UTC time

In [171]:
UTC_start = convert_UTC(date,t)
print(UTC_start)

530
20240211T202848530Z


### Create line to write in the Timeline csv file 

In [16]:
header_0 = ['Instrument','','Temps','','Geometrie','','','Position','','','Texte','Variables','']
header_1 = ['Index','Objet','T_0','T_F','X','Y','Z','Latitude','Longitude','Elevation','Commentaire','Facq','Theta']



In [175]:
Z = str(float(data['OSD.height [ft]'][idx_start])*0.308)
latitude = data['OSD.latitude'][idx_start]
longitude = data['OSD.longitude'][idx_start]
elevation = str(float(data['OSD.altitude [ft]'][idx_start])*0.308)
theta = data['GIMBAL.pitch'][idx_start]

line_csv = ['','',UTC_start,'','','',Z,latitude,longitude,elevation,'','30',theta]
print(line_csv)

['', '', '20240211T202848530Z', '', '', '', '15.1536', 48.2517258135521, -70.0911424895404, '15.092', '', '30', -90.0]


In [172]:
a = '20.08'
b = float(a)
print(b)

20.08


In [67]:
w = '02/01/2024'
w_length = len(w)
year = w[w_length - 4:]
day = w[w_length - 7 : w_length - 5]

if w_length == 10 :
    month = w[:2]
else :
    month = '0' + w[0]

f = year + month + day
    
print(f)

20240201


In [14]:
facq = 30
line_csv = get_timeline_row(data,idx_start,'',facq)

print(line_csv)


2/11/2024
530
['', '', '20240211T202848530Z', '', '', '', 15.1536, 48.2517258135521, -70.0911424895404, 15.092, '', 30, -90.0]


In [18]:
filename = 'G:/Rimouski_2024/Data/2024/0211/Drones/bernache/test.csv'


with open(filename, 'w', newline="") as file:
    csvwriter = csv.writer(file) # 1. create a csvwriter object
    csvwriter.writerow(header_0) # 2. write the header
    csvwriter.writerow(header_1) # 3. write second header
    csvwriter.writerow(line_csv) # 4. write data
    
    

In [161]:
# convert a time year, month, day, hour, min, sec, microsec to a timme in seconds since the epoch
t = dt.datetime(2024,10,1,10,2,3).timestamp()

# convert a time to UTC time 
UTC_t = time.gmtime(t)
print(UTC_t)

time.struct_time(tm_year=2024, tm_mon=10, tm_mday=1, tm_hour=14, tm_min=2, tm_sec=3, tm_wday=1, tm_yday=275, tm_isdst=0)


In [162]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9
